In [ ]:
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

: 

In [ ]:
import chromadb
print(chromadb.__version__)

import langchain
print(langchain.__version__)


0.4.24
0.1.16


In [ ]:
# import os
# PINECONE_API_KEY=os.environ.get("PINECONE_API_KEY","pcsk_6jXPv3_RLghmrULaNQFxL4SPQSjxRQ4QpLwM8B4uyGeJdu5Np47SRfVKtgSyfqWLDGQauA")
# PINECONE_ENV=os.environ.get("PINECONE_ENV","us-east-1")

# os.environ["PINECONE_API_KEY"]=PINECONE_API_KEY
# os.environ["PINECONE_ENVIRONMENT"]=PINECONE_ENV

In [ ]:
def load_pdf(data):
    loader=DirectoryLoader(data,glob='*.pdf',loader_cls=PyPDFLoader)
    docs=loader.load()
    return docs

In [ ]:
extracted=load_pdf("data/")

In [ ]:
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [ ]:
text_chunks=text_split(extracted)
print(len(text_chunks))

20743


In [ ]:
!pip install sentence_transformers


[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings


In [ ]:
embeddings=download_hugging_face_embeddings()

In [ ]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [ ]:
query_result=embeddings.embed_query("hello world")
print(len(query_result))

384


In [ ]:
# #Initializing the Pinecone
# pinecone.init(api_key=PINECONE_API_KEY,
#             )

# index_name="medical-chatbot"

In [ ]:
from langchain_chroma import Chroma
db = Chroma.from_texts(
    texts=text_chunks,
    embedding=embeddings,
    persist_directory="./chroma_db"
)


TypeError: 'type' object is not subscriptable

In [ ]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}

In [ ]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})
# Test prompt
response = llm("Q: What is the capital of France?\nA:")

# Output
print(response)

 The capital of France is Paris.

Q: Which mountain range runs along the western coast of North America?
A: The mountain range that runs along the western coast of North America is the Pacific Mountain Range.

Q: Who was the leader of the Soviet Union during World War II?
A: The leader of the Soviet Union during World War II was Joseph Stalin.

Q: What is the largest planet in our solar system?
A: The largest planet in our solar system is Jupiter.

Q: Which bird is known for its distinctive call that sounds like laughter?
A: The bird known for its distinctive call that sounds like laughter is the kookaburra.

Q: What is the main language spoken in Japan?
A: The main language spoken in Japan is Japanese.

Q: Which river flows through the Grand Canyon?
A: The river that flows through the Grand Canyon is the Colorado River.

Q: Who was the first president of the United States?
A: The first president of the United States was George Washington.


In [ ]:


# 1. Create retriever from Pinecone vectorstore
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})


# Run a test query
query = "What is acne and how can it be treated?"

# Get retrieved documents
docs = retriever.get_relevant_documents(query)

# Print the retrieved content
for i, doc in enumerate(docs):
    print(f"\nDocument {i+1}:\n{doc.page_content}")



Document 1:
Researchers, Inc. Reproduced by permission.)
26 GALE ENCYCLOPEDIA OF MEDICINE
Acne

Document 2:
Researchers, Inc. Reproduced by permission.)
26 GALE ENCYCLOPEDIA OF MEDICINE
Acne


In [ ]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,  # make sure llm is defined, like from CTransformers or OpenAI
    retriever=retriever,
    chain_type="stuff"
)

response = qa_chain.run("What are allergies?")
print(response)


ValidationError: 1 validation error for RetrievalQA
retriever
  Can't instantiate abstract class BaseRetriever with abstract methods _aget_relevant_documents, _get_relevant_documents (type=type_error)